# **Principal Component Analysis (PCA)**

# **Global Constants**

In [ ]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Teaching/2022-23/2022-23-BDC/datasets"
DATASET_URL = "https://github.com/gtolomei/big-data-computing/raw/master/datasets/mnist-train.csv.bz2"
GDRIVE_DATASET_FILE = GDRIVE_DATA_DIR + "/" + DATASET_URL.split("/")[-1]

RANDOM_SEED = 42 # for reproducibility

# **Spark + Google Colab Setup**

## **1.** Install PySpark and related dependencies

In [ ]:
!pip install pyspark
# Alternatively, if you want to install a specific version of pyspark:
#!pip install pyspark==3.2.1
!pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME


## **2.** Import useful Python packages

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## **3.** Create Spark context

In [ ]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## **4.** Create <code>ngrok</code> tunnel to check the Spark UI

In [ ]:
# Install ngrok
!pip install pyngrok

In [ ]:
# Be sure you create your own account at https://dashboard.ngrok.com/login and replace the token string below with yours
!ngrok authtoken 2MamtHU170jRTFqA7ai0WZFniY9_825Vvne665fhVDZdRKNHT # Replace with your own authtoken

In [ ]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [ ]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

## **5.** Link Colab to our Google Drive

In [ ]:
# Point Colaboratory to our Google Drive

from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

## **6.** Check everything is ok

In [ ]:
spark

In [ ]:
sc._conf.getAll()

# **Data Acquisition**

## **MNIST Dataset**

[MNIST](http://yann.lecun.com/exdb/mnist/) ("Modified National Institute of Standards and Technology") is the de facto "Hello World" dataset of computer vision. Since its release in 1999, this classic dataset of handwritten digit images has served as the basis for benchmarking classification algorithms.

### Download dataset file from URL directly to our Google Drive.

In [ ]:
def get_data(dataset_url, dest, chunk_size=1024):
  response = requests.get(dataset_url, stream=True)
  if response.status_code == 200:
    with open(dest, "wb") as file:
      for block in response.iter_content(chunk_size=chunk_size): 
        if block: 
          file.write(block)

In [ ]:
print("Retrieving dataset from URL: {} ...".format(DATASET_URL))
get_data(DATASET_URL, GDRIVE_DATASET_FILE)
print("Dataset successfully retrieved and stored at: {}".format(GDRIVE_DATASET_FILE))

### Read dataset file into a Spark Dataframe

In [ ]:
mnist_df = spark.read.load(GDRIVE_DATASET_FILE, 
                         format="csv", 
                         sep=",", 
                         inferSchema="true", 
                         header="true"
                         )

### Check the shape of the loaded dataset, i.e., number of rows and columns

In [ ]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(mnist_df.count(), len(mnist_df.columns)))

### Print out the schema of the loaded dataset

In [ ]:
mnist_df.printSchema()

### Dataset Shape and Schema

The dataset has 785 columns. The first column, called `label`, is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated 28-by-28 pixels image, i.e., a 784-dimensional vector.

Each pixel column in the dataset has a name like <code>pixel**k**</code>, where **`k`** is an integer between 0 and 783, inclusive. 
To locate this pixel on the image, suppose that we have decomposed **`k`** as **`k`** = `i * 28 + j`, where `i` and `j` are integers between 0 and 27, inclusive. Then <code>pixel**k**</code> is located on row `i` and column `j` of a 28 x 28 matrix, (indexing by zero).

For example, <code>pixel**31**</code> indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

### Display the first 5 rows of the dataset

In [ ]:
mnist_df.show(5)

# **Data Preprocessing**

### Assembling Features into a single column

We make use of the `VectorAssembler` transformer, which combines a given list of columns into a single vector column.

In [ ]:
columns = ['pixel{:d}'.format(k) for k in range(784)]

In [ ]:
columns[:5]

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=columns, 
                            outputCol="features")

In [ ]:
mnist_df = assembler.transform(mnist_df)
mnist_df.select("features", "label").show(truncate=False)

### Eventually, retain only 2 columns: assembled features and label

In [ ]:
mnist_df = mnist_df.select("features", "label")

# **Principal Component Analysis (PCA)**

In [ ]:
mnist_df.show(5)

## **Standardize features**

As we know, PCA is highly sensitive to feature scale. Let's standardize each feature so that each feature value $x$ is transformed into $z$, as follows:
$$
  z = \frac{x-\mu}{\sigma}
$$
where $\mu$ is the sample mean of the feature, and $\sigma$ is the unbiased sample standard deviation (computed from all the observations).
In such a way, each $z$ will have $0$-mean and $1$-standard-deviation.

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", 
                        outputCol="std_features",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(mnist_df)

# Normalize each feature to have unit standard deviation.
mnist_df = scalerModel.transform(mnist_df)

In [ ]:
mnist_df.show(5, truncate=False)

## **Run PCA**

In [ ]:
K = 10 # number of principal components to extract

In [ ]:
from pyspark.ml.feature import PCA

pca_model = PCA(k=K, inputCol="std_features", outputCol="pca_features")
pca_features = pca_model.fit(mnist_df)
pca_mnist_df = pca_features.transform(mnist_df).cache()

In [ ]:
pca_mnist_df.show(5)

## **Plotting**

### Show the proportion of the data variance carried by each principal component (eigenvector)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(8,6))
_ = sns.barplot(x=[i for i in range(K)], 
                y=pca_features.explainedVariance.values, # `explainedVariance` returns the distribution of variance across eigenvectors, i.e., lambda_i/sum lambda_i
                ax=ax)
_ = ax.set_xlabel("Eigenvalues", labelpad=16, fontsize=16)
_ = ax.set_ylabel("Proportion of Variance", fontsize=16)
_ = ax.set_xticklabels(["lambda_{:d}".format(i) for i in range(K)], rotation=45)

### Transform the PySpark DataFrame into Pandas' DataFrame

In [ ]:
pca_mnist_pdf = pca_mnist_df.toPandas()

In [ ]:
pca_mnist_pdf.head()

### Plot data projected on the 2 principal components

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,8))
_ = plt.scatter(
    pca_mnist_pdf.pca_features.map(lambda x: x[0]), 
    pca_mnist_pdf.pca_features.map(lambda x: x[1]),
    c=pca_mnist_pdf.label, 
    edgecolor='none', 
    alpha=0.8,
    cmap=plt.cm.get_cmap('tab10', 10),
    axes=ax
    )
_ = ax.set_xlabel('PCA 1', labelpad=20, fontsize = 16)
_ = ax.set_ylabel('PCA 2', fontsize = 16)
plt.colorbar();

# **K-means Clustering**

### Function used for running K-means

In [ ]:
def k_means(dataset, 
            n_clusters, 
            distance_measure="euclidean", 
            max_iter=20, 
            features_col="features", 
            prediction_col="cluster", 
            random_seed=RANDOM_SEED):
  
  from pyspark.ml.clustering import KMeans

  print("""Training K-means clustering using the following parameters: 
  - K (n. of clusters) = {:d}
  - max_iter (max n. of iterations) = {:d}
  - distance measure = {:s}
  - random seed = {:d}
  """.format(n_clusters, max_iter, distance_measure, random_seed))
  # Train a K-means model
  kmeans = KMeans(featuresCol=features_col, 
                   predictionCol=prediction_col, 
                   k=n_clusters, 
                   initMode="k-means||", 
                   initSteps=5, 
                   tol=0.000001, 
                   maxIter=max_iter, 
                   seed=random_seed, 
                   distanceMeasure=distance_measure)
  model = kmeans.fit(dataset)

  # Make clusters
  clusters_df = model.transform(dataset)

  return model, clusters_df

### Function used to evaluate obtained clusters

In [ ]:
def evaluate_k_means(clusters, 
                     metric_name="silhouette", 
                     distance_measure="squaredEuclidean", # cosine
                     prediction_col="cluster"
                     ):
  
  from pyspark.ml.evaluation import ClusteringEvaluator
  
  # Evaluate clustering by computing Silhouette score
  evaluator = ClusteringEvaluator(metricName=metric_name,
                                  distanceMeasure=distance_measure, 
                                  predictionCol=prediction_col
                                  )

  return evaluator.evaluate(clusters)

### Run K-means by calling the function above

In [ ]:
model, clusters_df = k_means(pca_mnist_df, 
                             10,
                             distance_measure="euclidean",
                             max_iter=1000, 
                             features_col="pca_features"
                             )

In [ ]:
evaluate_k_means(clusters_df)

In [ ]:
clusters_df.show(5)

In [ ]:
clusters_df.groupBy("cluster").count().sort("cluster").show()

In [ ]:
# Get unique values in the grouping column
clusters = sorted([x[0] for x in clusters_df.select("cluster").distinct().collect()])
print("Cluster IDs: [{:s}]".format(", ".join([str(c) for c in clusters])))

# Create a filtered DataFrame for each group in a list comprehension
cluster_list = [clusters_df.where(clusters_df.cluster == x) for x in clusters]

# Show the results
for x_id, x in enumerate(cluster_list):
  print("Showing the first 10 records of cluster ID #{:d}".format(x_id))
  x.select(["cluster", "label"]).show(10, truncate=False)